In [9]:
import numpy as np

from HRED_v2 import HRED
from squad_load import get_glove_train_embs, get_squad_train_voc, load_squad_train

from itertools import chain

In [2]:
squad_path = '/pio/data/data/squad/'
glove_path = '/pio/data/data/glove_vec/6B/'

data = load_squad_train(squad_path)
i_to_w, w_to_i, voc_size = get_squad_train_voc(squad_path)
glove_embs = get_glove_train_embs(squad_path, glove_path)

def get_w(idx):
    return i_to_w[idx]

In [4]:
def trim_data(data, trim=100):
    return [d for d in data if max(map(len, d[1])) <= trim]

data_trimmed = trim_data(data)
len(data), len(data_trimmed)

(87599, 86789)

In [3]:
hred_net = HRED(voc_size=voc_size,
                emb_size=300,
                lv1_rec_size=300,
                lv2_rec_size=300,
                out_emb_size=300,
                emb_init=glove_embs)

Building the model...
Compiling theano functions...
Done


In [7]:
ans = hred_net.get_output(data[:12], 3)
ans[0]

[array([ 0.01092721,  0.00748011,  0.00762079,  0.01578316,  0.01117014,
         0.01161463,  0.02582562,  0.01773244,  0.00615017], dtype=float32),
 array([ 0.00799526,  0.00572318,  0.00518265,  0.00791706,  0.01088359,
         0.02770218,  0.02637224,  0.00865242,  0.0114931 ,  0.02562324,
         0.02522738,  0.01880244,  0.01147158,  0.0155955 ,  0.01875451,
         0.00431506], dtype=float32),
 array([ 0.00283376,  0.00202473,  0.00628567,  0.00842987,  0.00693043,
         0.00607813,  0.00879546,  0.01265835,  0.01486384,  0.00704044,
         0.0058486 ,  0.00434419,  0.00848916,  0.02093938,  0.02297711,
         0.01494346,  0.01414375,  0.00701558,  0.01150942,  0.01826271,
         0.00650786,  0.0057557 ,  0.0118857 ,  0.0068161 ,  0.02206977,
         0.02071751,  0.01085666,  0.03032649,  0.00578328,  0.00204773], dtype=float32),
 array([ 0.00315659,  0.00204183,  0.00317063,  0.0037629 ,  0.00659333,
         0.00453729,  0.00695357,  0.0258143 ,  0.01929855,  0.01

In [12]:
list(chain(*ans[8]))[46]

0.05232447

In [11]:
get_w(list(chain(*data[8][1][1:]))[45])

u'september'

In [10]:
list(reversed(np.array(list(chain(*ans[8]))).argsort()))

[46,
 23,
 45,
 74,
 7,
 19,
 49,
 8,
 6,
 126,
 10,
 24,
 28,
 80,
 9,
 29,
 54,
 43,
 31,
 37,
 4,
 42,
 123,
 82,
 11,
 30,
 111,
 94,
 16,
 35,
 81,
 15,
 50,
 148,
 62,
 78,
 53,
 91,
 204,
 33,
 13,
 12,
 105,
 21,
 20,
 155,
 152,
 122,
 36,
 158,
 125,
 203,
 217,
 5,
 22,
 84,
 14,
 87,
 27,
 34,
 3,
 85,
 107,
 222,
 186,
 79,
 25,
 110,
 128,
 56,
 175,
 60,
 127,
 131,
 52,
 117,
 63,
 149,
 112,
 47,
 26,
 66,
 55,
 2,
 32,
 86,
 197,
 171,
 151,
 61,
 108,
 83,
 137,
 119,
 124,
 48,
 67,
 166,
 201,
 134,
 239,
 1,
 172,
 44,
 236,
 90,
 233,
 106,
 77,
 240,
 104,
 176,
 71,
 141,
 75,
 0,
 227,
 142,
 101,
 109,
 198,
 147,
 202,
 154,
 73,
 207,
 18,
 235,
 226,
 93,
 65,
 59,
 150,
 120,
 194,
 113,
 195,
 205,
 121,
 118,
 153,
 57,
 232,
 39,
 17,
 216,
 132,
 41,
 140,
 247,
 116,
 178,
 64,
 97,
 179,
 100,
 58,
 224,
 76,
 170,
 70,
 69,
 51,
 243,
 103,
 133,
 169,
 136,
 218,
 237,
 238,
 143,
 72,
 174,
 167,
 102,
 225,
 177,
 200,
 89,
 181,
 114,
 146,
 20

In [ ]:
# Zwykły trening nie robił nic: wartości zwracane przez sieć dla zbioru uczącego nie miały żadnego sensu.
# Największe prawdopodobieństwa były prawie niezależne od pytania, miało ono mały wpływ na to, co się dzieje.
# Dla kosztu weighted_bin_ce mocno skrzywionego w stronę sytuacji t=1 wyszło podobnie.
#
# Następny krok: użyć zanurzeń GloVe, dodać cechy z https://arxiv.org/abs/1703.04816
#
# UPDATE:
# Obie cechy podobieństwa kontekstu do pytania dodane.
# W glove.6B jest około 70% słów ze zbioru uczącego SQuAD.
# W tej chwili te wektory biorę z glove.6B.300d, a pozostałym daję losowy init, 
# po czym wszystkie zanurzenia są dalej uczone.

In [6]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # weighted cost, random init

Done 20 batches in 7.96s	training loss:	0.813393
Done 40 batches in 16.89s	training loss:	0.618499
Done 60 batches in 25.51s	training loss:	0.537964
Done 80 batches in 33.72s	training loss:	0.476587
Done 100 batches in 41.79s	training loss:	0.436442
Done 120 batches in 48.56s	training loss:	0.419979
Done 140 batches in 57.10s	training loss:	0.409618
Done 160 batches in 65.07s	training loss:	0.408088
Done 180 batches in 74.05s	training loss:	0.397444
Done 200 batches in 81.68s	training loss:	0.401948
Done 220 batches in 90.27s	training loss:	0.401396
Done 240 batches in 98.95s	training loss:	0.410639
Done 260 batches in 107.43s	training loss:	0.418680
Done 280 batches in 115.18s	training loss:	0.427639
Done 300 batches in 123.33s	training loss:	0.426674
Done 320 batches in 131.00s	training loss:	0.422292
Done 340 batches in 139.84s	training loss:	0.424456
Done 360 batches in 148.91s	training loss:	0.415105
Done 380 batches in 156.67s	training loss:	0.416838
Done 400 batches in 163.90s	t

KeyboardInterrupt: 

In [5]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # normal binary cross-entropy with random init

Done 20 batches in 2.25s	training loss:	0.228349
Done 40 batches in 5.03s	training loss:	0.154742
Done 60 batches in 7.65s	training loss:	0.126147
Done 80 batches in 10.05s	training loss:	0.106807
Done 100 batches in 12.52s	training loss:	0.095184
Done 120 batches in 14.64s	training loss:	0.090502
Done 140 batches in 17.25s	training loss:	0.087078
Done 160 batches in 19.51s	training loss:	0.085557
Done 180 batches in 22.31s	training loss:	0.082334
Done 200 batches in 24.40s	training loss:	0.082256
Done 220 batches in 26.96s	training loss:	0.081215
Done 240 batches in 29.60s	training loss:	0.082376
Done 260 batches in 32.51s	training loss:	0.083784
Done 280 batches in 34.97s	training loss:	0.084956
Done 300 batches in 37.36s	training loss:	0.084330
Done 320 batches in 39.52s	training loss:	0.083391
Done 340 batches in 42.24s	training loss:	0.083563
Done 360 batches in 44.99s	training loss:	0.081207
Done 380 batches in 47.14s	training loss:	0.081512
Done 400 batches in 49.09s	training lo

KeyboardInterrupt: 

In [6]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # normal binary cross-entropy with glove init

Done 20 batches in 2.62s	training loss:	0.196862
Done 40 batches in 5.85s	training loss:	0.147509
Done 60 batches in 8.90s	training loss:	0.121683
Done 80 batches in 11.70s	training loss:	0.103534
Done 100 batches in 14.59s	training loss:	0.092342
Done 120 batches in 17.06s	training loss:	0.088309
Done 140 batches in 20.10s	training loss:	0.084997
Done 160 batches in 22.75s	training loss:	0.083670
Done 180 batches in 26.01s	training loss:	0.080552
Done 200 batches in 28.45s	training loss:	0.080706
Done 220 batches in 31.44s	training loss:	0.080079
Done 240 batches in 34.52s	training loss:	0.081357
Done 260 batches in 37.91s	training loss:	0.084765
Done 280 batches in 40.77s	training loss:	0.086710
Done 300 batches in 43.56s	training loss:	0.086035
Done 320 batches in 46.09s	training loss:	0.084813
Done 340 batches in 49.25s	training loss:	0.084861
Done 360 batches in 52.45s	training loss:	0.082334
Done 380 batches in 54.97s	training loss:	0.082462
Done 400 batches in 57.25s	training lo

0.088305816713791049

In [ ]:
hred_net.save_params()